# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

## Code

In [6]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [7]:
fp = r"data\2014_LoL_esports_match_data_from_OraclesElixir.csv"

with open(fp, 'r') as f:
    columns = (f.readline())
columns

'gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,participantid,side,position,playername,playerid,teamname,teamid,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xp

In [8]:
# Reading the data
from tqdm.notebook import tqdm

pd.set_option('display.max_columns', None)
usecols = ['gameid','datacompleteness', 'league', 'year', 'date', 'game', 'patch', 
           'side', 'position', 'playername', 'teamname', 'champion', 'gamelength', 'result',
           'elementaldrakes', 'opp_elementaldrakes', 'elders', 'opp_elders', 'firstherald', 'barons', 'opp_barons', 'firstbaron',
           'golddiffat10', 'xpdiffat10', 'csdiffat10', 
           'golddiffat15', 'xpdiffat15', 'csdiffat15',]
                 

directory = r'data'
df = pd.DataFrame()
for filename in tqdm(os.listdir(directory)):
    new_df = pd.read_csv(os.path.join(directory, filename), usecols=usecols)
    df = pd.concat([df, new_df])
df

  0%|          | 0/10 [00:00<?, ?it/s]

,gameid,datacompleteness,league,year,date,game,patch,side,position,playername,teamname,champion,gamelength,result,elementaldrakes,opp_elementaldrakes,elders,opp_elders,firstherald,firstbaron,barons,opp_barons,golddiffat10,xpdiffat10,csdiffat10,golddiffat15,xpdiffat15,csdiffat15
0,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,top,sOAZ,Fnatic,Trundle,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.0,-87.0,0.0,49.0,-560.0,-1.0
1,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,jng,Cyanide,Fnatic,Vi,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1102.0,-1425.0,-27.0,-1107.0,-703.0,-36.0
2,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,mid,xPeke,Fnatic,Orianna,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-325.0,-87.0,5.0,-145.0,-148.0,7.0
3,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,bot,Rekkles,Fnatic,Jinx,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,977.0,840.0,27.0,1588.0,998.0,22.0
4,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,sup,YellOwStaR,Fnatic,Annie,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.0,296.0,-9.0,887.0,476.0,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129343,ESPORTSTMNT02_3251457,complete,NEXO,2024,2023-11-13 20:32:25,1.0,13.21,Red,mid,Tirex,Universae Instituto FP,Ahri,1910,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-105.0,177.0,-8.0,677.0,736.0,-2.0
129344,ESPORTSTMNT02_3251457,complete,NEXO,2024,2023-11-13 20:32:25,1.0,13.21,Red,bot,Alonshot,Universae Instituto FP,Aphelios,1910,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-204.0,249.0,0.0,-956.0,-16.0,-11.0
129345,ESPORTSTMNT02_3251457,complete,NEXO,2024,2023-11-13 20:32:25,1.0,13.21,Red,sup,Boohis,Universae Instituto FP,Rakan,1910,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-104.0,63.0,14.0,107.0,497.0,17.0
129346,ESPORTSTMNT02_3251457,complete,NEXO,2024,2023-11-13 20:32:25,1.0,13.21,Blue,team,NaN,Erfolg Esports,NaN,1910,0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,-246.0,-1170.0,-23.0,-1386.0,-2270.0,-41.0


In [9]:
# Data Cleaning
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Remove games after October 2023 (the current league is still ongoing)
time_cutoff = pd.to_datetime('2023-10-01')
df = df[df['date'] < time_cutoff]

# Filter out incomplete data, and fill in NA player names with 'unknown player'
df = df[df['datacompleteness'] == 'complete']
df['playername'] = df['playername'].fillna('unknown player')

# Remove team summary rows
#df = df.query('position != "team"')df


In [10]:
df[df['league'] == 'LPL'].sort_values('golddiffat15')

,gameid,datacompleteness,league,year,date,game,patch,side,position,playername,teamname,champion,gamelength,result,elementaldrakes,opp_elementaldrakes,elders,opp_elders,firstherald,firstbaron,barons,opp_barons,golddiffat10,xpdiffat10,csdiffat10,golddiffat15,xpdiffat15,csdiffat15
60431,7380-10000,complete,LPL,2021,2021-04-06 11:28:59,3.0,11.06,Red,team,unknown player,Suning,NaN,1143,0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,-7026.0,-4006.0,-67.0,-11850.0,-6116.0,-116.0
39538,7320-9752,complete,LPL,2021,2021-03-04 12:01:00,2.0,11.03,Blue,team,unknown player,Oh My God,NaN,1216,0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,-4248.0,-2963.0,-49.0,-11643.0,-8079.0,-108.0
58775,7376-9986,complete,LPL,2021,2021-04-02 11:04:19,3.0,11.06,Red,team,unknown player,LNG Esports,NaN,1040,0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,-3516.0,-3601.0,-60.0,-11513.0,-7627.0,-106.0
73582,5391-6600,complete,LPL,2019,2019-07-29 10:10:38,2.0,9.14,Blue,team,unknown player,Victory Five,NaN,1144,0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0.0,-3545.0,-2270.0,-1.0,-11078.0,-7123.0,-51.0
56530,6230-7993,complete,LPL,2020,2020-06-20 10:11:00,2.0,10.12,Blue,team,unknown player,Dominus Esports,NaN,1569,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,-3907.0,-2225.0,-69.0,-9472.0,-5261.0,-128.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56531,6230-7993,complete,LPL,2020,2020-06-20 10:11:00,2.0,10.12,Red,team,unknown player,Oh My God,NaN,1569,1,NaN,NaN,0.0,0.0,1.0,1.0,1.0,0.0,3907.0,2225.0,69.0,9472.0,5261.0,128.0
73583,5391-6600,complete,LPL,2019,2019-07-29 10:10:38,2.0,9.14,Red,team,unknown player,Top Esports,NaN,1144,1,NaN,NaN,0.0,0.0,1.0,NaN,0.0,0.0,3545.0,2270.0,1.0,11078.0,7123.0,51.0
58774,7376-9986,complete,LPL,2021,2021-04-02 11:04:19,3.0,11.06,Blue,team,unknown player,Suning,NaN,1040,1,NaN,NaN,0.0,0.0,1.0,NaN,0.0,0.0,3516.0,3601.0,60.0,11513.0,7627.0,106.0
39539,7320-9752,complete,LPL,2021,2021-03-04 12:01:00,2.0,11.03,Red,team,unknown player,Top Esports,NaN,1216,1,NaN,NaN,0.0,0.0,1.0,NaN,0.0,0.0,4248.0,2963.0,49.0,11643.0,8079.0,108.0


In [11]:
adcs = df.query('position == "bot"')
adcs['golddiffat15'].describe()

count    120540.000000
mean          0.000000
std        1137.959267
min       -7659.000000
25%        -673.000000
50%           0.000000
75%         673.000000
max        7659.000000
Name: golddiffat15, dtype: float64

### Framing the Problem

In [12]:
# TODO

### Baseline Model

In [13]:
# TODO
df

,gameid,datacompleteness,league,year,date,game,patch,side,position,playername,teamname,champion,gamelength,result,elementaldrakes,opp_elementaldrakes,elders,opp_elders,firstherald,firstbaron,barons,opp_barons,golddiffat10,xpdiffat10,csdiffat10,golddiffat15,xpdiffat15,csdiffat15
0,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,top,sOAZ,Fnatic,Trundle,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.0,-87.0,0.0,49.0,-560.0,-1.0
1,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,jng,Cyanide,Fnatic,Vi,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1102.0,-1425.0,-27.0,-1107.0,-703.0,-36.0
2,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,mid,xPeke,Fnatic,Orianna,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-325.0,-87.0,5.0,-145.0,-148.0,7.0
3,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,bot,Rekkles,Fnatic,Jinx,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,977.0,840.0,27.0,1588.0,998.0,22.0
4,TRLH3/33,complete,EU LCS,2014,2014-01-14 17:52:02,1.0,3.15,Blue,sup,YellOwStaR,Fnatic,Annie,1924,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337.0,296.0,-9.0,887.0,476.0,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122635,ESPORTSTMNT05_3242449,complete,CDF,2023,2023-09-29 17:15:44,3.0,13.18,Red,mid,Peng,Solary,Lucian,1550,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-264.0,103.0,16.0,-243.0,159.0,14.0
122636,ESPORTSTMNT05_3242449,complete,CDF,2023,2023-09-29 17:15:44,3.0,13.18,Red,bot,TakeSet,Solary,Kai'Sa,1550,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,245.0,-154.0,12.0,434.0,173.0,18.0
122637,ESPORTSTMNT05_3242449,complete,CDF,2023,2023-09-29 17:15:44,3.0,13.18,Red,sup,Steeelback,Solary,Alistar,1550,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-549.0,-401.0,0.0,-604.0,-94.0,0.0
122638,ESPORTSTMNT05_3242449,complete,CDF,2023,2023-09-29 17:15:44,3.0,13.18,Blue,team,unknown player,Team du Sud,NaN,1550,1,3.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,2679.0,2015.0,-5.0,2486.0,1413.0,0.0


In [16]:

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
df2 = df.copy().fillna(0)
X_train, X_test, y_train, y_test = train_test_split(df2.drop(columns=['result']), df2['result'], test_size=0.2)
preproc = ColumnTransformer(transformers=[
    ('onehot', OneHotEncoder(), ['league', 'side']),
    ('passthrough', 'passthrough', ['golddiffat15', 'xpdiffat15', 'elders', 'barons', 'firstherald', 'firstbaron']),
], remainder='drop')

model = Pipeline([
    ('preprocessing', preproc),
    ('classification', RandomForestClassifier(max_depth=5))  # Set the max_depth parameter here
])

model.fit(X_train, y_train)
base= model.predict(X_test)
base

array([0, 0, 0, ..., 0, 1, 1], dtype=int64)

In [22]:
accuracy = 1- np.abs(base - y_test).sum() / len(y_test)
accuracy

0.6847421069732393

In [27]:
TP = ((base == 1) & (y_test == 1)).sum()
TN = ((base == 0) & (y_test == 0)).sum()
(TP, TN)
FP = ((base == 1) & (y_test == 0)).sum()
FN = ((base == 0) & (y_test == 1)).sum()
(TP, FP, TN, FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)
accuracy

0.6847421069732395

### Final Model

In [ ]:
# TODO

### Fairness Analysis

In [ ]:
# TODO